# Steamer Historical Weekly Logs <br>
This should only need to be run once ever <br>
It takes the Steamer Weekly Log files, which are massive csvs with a ROS projection for each player each week from 2014-2022 and then makes daily projection files from it <br>
Daily projections use last weekly projection available <br>
Might need to use this rest of season if I can't scrape FanGraphs <br>

May have gone too far producing model inputs here. Make compatible with A5. Stats

In [1]:
import pandas as pd
import numpy as np
import os
import datetime

baseball_path = r"C:\Users\james\Documents\MLB\Data"

In [2]:
# These are the stats we need to use to project
pitcher_list = ['p_L', 'H/9', 'HR/9', 'K/9', 'BB/9']
batter_list = ['b_L', 'b1_rate', 'b2_rate', 'b3_rate', 'hr_rate', 'bb_rate', 'hbp_rate', 'so_rate']

# These are the stats we want to keep
bat_list = ['proj_date', 'mlbamid', 'steamerid', 'firstname', 'lastname', 'Team', 'position', 'bats', 'PA', 'IBB', 'NIBB', 'BB', 'K', 'HBP', 'H', '2B', '3B', 'HR', 'OBP', 'SLG', 'wOBA', 'SB', 'CS']
pitch_list = ['proj_date', 'mlbamid', 'steamerid', 'firstname', 'lastname', 'Throws', 'IP', 'G', 'GS', 'K9', 'BB9', 'H', 'HR']

In [16]:
# Read in weekly log
hitters = pd.read_csv(os.path.join(baseball_path, "Y. One Time Use", "Steamer Weekly Log", "steamer_hitters_weekly_log.csv"))

# Make a function for this?
# Seiya Suzuki wasn't appearing, so I copied his next week projection
duplicate1 = hitters.query('firstname == "Seiya"').query('lastname == "Suzuki"').query('proj_date == "2022-04-15"')
duplicate1['proj_date'] = duplicate1['proj_date'].replace("2022-04-15", "2022-04-08")

duplicate2 = hitters.query('firstname == "Christian"').query('lastname == "Bethancourt"').query('proj_date == "2022-04-22"')
duplicate2['proj_date'] = duplicate2['proj_date'].replace("2022-04-22", "2022-04-15")

hitters = pd.concat([hitters, duplicate1, duplicate2])
hitters.head()


,proj_year,postseason,proj_date,mlbamid,steamerid,firstname,lastname,Team,position,bats,...,SLG,wOBA,SB,CS,wSB,UBR,BaseRunning,UZR,R,RBI
0,2014,0,2014-06-17,607680.0,12434,Kevin,Pillar,TOR,CF,R,...,0.374338,0.298854,2.9539,1.6472,-0.0654,0.109,0.0436,0.169,9.4540,9.0755
1,2014,0,2014-06-17,493128.0,13047,Munenori,Kawasaki,TOR,SS,L,...,0.312114,0.283956,0.0187,0.0089,0.0000,0.000,0.0000,0.000,0.0994,0.0776
2,2014,0,2014-06-17,136660.0,1392,Mark,DeRosa,TOR,2b,R,...,0.350845,0.300232,0.0070,0.0043,-0.0005,-0.002,-0.0029,0.000,0.1040,0.0965
3,2014,0,2014-06-17,408314.0,1736,Jose,Reyes,TOR,SS,B,...,0.421190,0.336366,15.0226,5.5845,0.7520,0.362,1.1136,-2.532,48.9535,34.2312
4,2014,0,2014-06-17,430832.0,1887,Jose,Bautista,TOR,RF,R,...,0.523212,0.393728,3.2033,1.7708,-0.1527,0.437,0.2842,-0.951,53.0616,52.8365


In [17]:
# # Read in weekly log
# hitters = pd.read_csv(os.path.join(baseball_path, "A5. Stats - 1. FanGraphs", "Batters", "Batters_FG_20230327.csv"))
# hitters['proj_year'] = 2023
# hitters['proj_date'] = '2023-03-27'

# # hitters['K'] = hitters['SO']
# hitters['position'] = hitters['POSITION']
# # Make a function for this
# # Seiya Suzuki wasn't appearing, so I copied his next week projection
# duplicate1 = hitters.query('firstname == "Seiya"').query('lastname == "Suzuki"').query('proj_date == "2022-04-15"')
# duplicate1['proj_date'] = duplicate1['proj_date'].replace("2022-04-15", "2022-04-08")

# duplicate2 = hitters.query('firstname == "Christian"').query('lastname == "Bethancourt"').query('proj_date == "2022-04-22"')
# duplicate2['proj_date'] = duplicate2['proj_date'].replace("2022-04-22", "2022-04-15")

# hitters = pd.concat([hitters, duplicate1, duplicate2])
# hitters.tail()


In [26]:


def clean_hitters(df, bat_list, year, start_month, start_day, end_month, end_day):
    # Keep relevant year and variables
    df = df[df['proj_year'] == year][bat_list]
    
    # These are just the variables we need in the model. This list is just for reference
    batter_list = ['b_L', 'b1_rate', 'b2_rate', 'b3_rate', 'hr_rate', 'bb_rate', 'hbp_rate', 'so_rate']
    
    df['playerid'] = df['steamerid']
    df.rename(columns={'K':'SO'}, inplace=True)
    
    df['Name'] = df['firstname'] + " " + df['lastname']
    
    # Create model inputs
    # df['b_L'] = (df['bats'] == "L").astype('int')
    # df['b1_rate'] = df['1B'] / df['PA']
    # df['b2_rate'] = df['2B'] / df['PA']
    # df['b3_rate'] = df['3B'] / df['PA']
    # df['hr_rate'] = df['HR'] / df['PA']
    # df['bb_rate'] = df['BB'] / df['PA']
    # df['hbp_rate'] = df['HBP'] / df['PA']
    # df['so_rate'] = df['K'] / df['PA']
    
    # Loop over all dates in range
    # Date ranges
    start_date = datetime.date(year, start_month, start_day)
    end_date = datetime.date(year, end_month, end_day)
    
    
    # Delta time
    delta = datetime.timedelta(days = 1)
    # Loop over time
    while (start_date <= end_date):

        # Destination file name
        file_date = str(start_date).replace("-", "")
        file_name = "Batters_FG_" + file_date + ".csv"
        print(df['proj_date'].unique())
        # If the date has a corresponding projection in the big dateframe
        if str(start_date) in df['proj_date'].unique():
            # Print date
            print(str(start_date))
            # Print if date has a unique projection
            print("Projection")
            
            # Make new dateframe
            weekly = df[df['proj_date'] == str(start_date)]
            
        # If it doesn't
        else:
            # Print date
            print(str(start_date))
            # Print if that date has a unique projection
            print("No projection")
            # Export existing dataframe
            
        # Export
        weekly.to_csv(os.path.join(baseball_path, "A5. Stats - 1. FanGraphs", "Batters", file_name), encoding='iso-8859-1')
        
        # Go to next date
        start_date += delta
    
    return df    

In [35]:
hitters_clean = clean_hitters(hitters, bat_list, 2021, 4, 2, 10, 12)

['2021-04-02' '2021-04-09' '2021-04-16' '2021-04-23' '2021-04-30'
 '2021-05-07' '2021-05-14' '2021-05-21' '2021-05-28' '2021-06-04'
 '2021-06-11' '2021-06-18' '2021-07-02' '2021-07-09' '2021-07-16'
 '2021-07-23' '2021-07-30' '2021-08-06' '2021-08-13' '2021-08-20'
 '2021-08-27' '2021-09-03' '2021-09-10' '2021-09-17' '2021-09-24'
 '2021-10-01']
2021-04-02
Projection
['2021-04-02' '2021-04-09' '2021-04-16' '2021-04-23' '2021-04-30'
 '2021-05-07' '2021-05-14' '2021-05-21' '2021-05-28' '2021-06-04'
 '2021-06-11' '2021-06-18' '2021-07-02' '2021-07-09' '2021-07-16'
 '2021-07-23' '2021-07-30' '2021-08-06' '2021-08-13' '2021-08-20'
 '2021-08-27' '2021-09-03' '2021-09-10' '2021-09-17' '2021-09-24'
 '2021-10-01']
2021-04-03
No projection
['2021-04-02' '2021-04-09' '2021-04-16' '2021-04-23' '2021-04-30'
 '2021-05-07' '2021-05-14' '2021-05-21' '2021-05-28' '2021-06-04'
 '2021-06-11' '2021-06-18' '2021-07-02' '2021-07-09' '2021-07-16'
 '2021-07-23' '2021-07-30' '2021-08-06' '2021-08-13' '2021-08-20'

In [44]:
# Read in weekly log
# pitchers = pd.read_csv(os.path.join(baseball_path, "FanGraphs", "Pitchers", "Pitchers_FG_20230327.csv"))

In [58]:
pitch_list = ['proj_date', 'mlbamid', 'steamerid', 'firstname', 'lastname', 'DBTeamId', 'Throws', 'IP', 'G', 'GS', 'K9', 'BB9', 'HR9', 'H']
def clean_pitchers(df, pitch_list, year, start_month, start_day, end_month, end_day):
    # This code is ugly here, just making some manual fixed
    # df['proj_year'] = 2023
    # df['K'] = df['SO']
    # df['proj_date'] = '2023-03-27'
    
    
    # Keep relevant year and variables
    df = df[df['proj_year'] == year][pitch_list]
    # These are just the variables we need in the model. This list is just for reference
    pitcher_list = ['p_L', 'H/9', 'HR/9', 'K/9', 'BB/9']
    
    # Create model inputs
#     df['p_L'] = (df['Throws'] == "L").astype('int')
#     df['H/9'] = df['H'] / df['IP'] * 9 
#     df['HR/9'] = df['HR9']
#     df['K/9'] = df['K9']
#     df['BB/9'] = df['BB9']
    
    df['playerid'] = df['steamerid']
    df['Name'] = df['firstname'] + " " + df['lastname']
    
    df.rename(columns={'K9':'K/9', 'BB9':'BB/9'}, inplace=True)
    
    # Loop over all dates in range
    # Date ranges
    start_date = datetime.date(year, start_month, start_day)
    end_date = datetime.date(year, end_month, end_day)
    
    # Delta time
    delta = datetime.timedelta(days = 1)
    
    # Loop over time
    while (start_date <= end_date):

        # Destination file name
        file_date = str(start_date).replace("-", "")
        file_name = "Pitchers_FG_" + file_date + ".csv"
        
        # If the date has a corresponding projection in the big dateframe
        if str(start_date) in df['proj_date'].unique():
            # Print date
            print(str(start_date))
            # Print if date has a unique projection
            print("Projection")
            
            # Make new dateframe
            weekly = df[df['proj_date'] == str(start_date)]
            
        # If it doesn't
        else:
            # Print date
            print(str(start_date))
            # Print if that date has a unique projection
            print("No projection")
            # Export existing dataframe
            
        # Export
        weekly.to_csv(os.path.join(baseball_path, "A5. Stats - 1. FanGraphs", "Pitchers", file_name), encoding='iso-8859-1')
        
        # Go to next date
        start_date += delta
    
    return df    

In [56]:
pitchers = pd.read_csv(os.path.join(baseball_path, "Y. One Time Use", "Steamer Weekly Log", "steamer_pitchers_weekly_log.csv"))


In [84]:
pitchers_clean = clean_pitchers(pitchers, pitch_list, 2020, 7, 24, 10, 27)

2020-07-24
Projection
2020-07-25
No projection
2020-07-26
No projection
2020-07-27
No projection
2020-07-28
No projection
2020-07-29
No projection
2020-07-30
No projection
2020-07-31
Projection
2020-08-01
No projection
2020-08-02
No projection
2020-08-03
No projection
2020-08-04
No projection
2020-08-05
No projection
2020-08-06
No projection
2020-08-07
Projection
2020-08-08
No projection
2020-08-09
No projection
2020-08-10
No projection
2020-08-11
No projection
2020-08-12
No projection
2020-08-13
No projection
2020-08-14
Projection
2020-08-15
No projection
2020-08-16
No projection
2020-08-17
No projection
2020-08-18
No projection
2020-08-19
No projection
2020-08-20
No projection
2020-08-21
Projection
2020-08-22
No projection
2020-08-23
No projection
2020-08-24
No projection
2020-08-25
No projection
2020-08-26
No projection
2020-08-27
No projection
2020-08-28
Projection
2020-08-29
No projection
2020-08-30
No projection
2020-08-31
No projection
2020-09-01
No projection
2020-09-02
No proj

In [83]:
pitchers['proj_date'].unique()

NameError: name 'batters' is not defined

In [82]:
pitchers.columns

Index(['proj_year', 'postseason', 'proj_date', 'mlbamid', 'steamerid',
       'firstname', 'lastname', 'IP', 'G', 'GS', 'start_percent', 'start_IP',
       'relief_IP', 'SV', 'League', 'DBTeamId', 'FBV', 'reliability', 'Throws',
       'age', 'Krate', 'BBrate', 'HBPrate', 'GBrate', 'FBrate', 'LDrate',
       'SIERA', 'BABIP', 'TBF', 'batters_per_IP', 'ERA', 'RA', 'K', 'BB', 'HR',
       'HBP', 'SH', 'GB', 'FB', 'LD', 'K9', 'BB9', 'HR9', 'H', 'ER', 'R',
       'WHIP', 'W', 'L'],
      dtype='object')